# Computing Embeddings of Recipes

Using the Ollama embedding tutorial here: https://ollama.com/blog/embedding-models
Note: Requires ChromaDB version 0.5.3 due to later versions crashing. 

In [1]:
import json
import os 
import ollama
import chromadb
import gc

In [2]:
# load the recipe JSONs as `documents`
json_recipe_files = [f for f in os.listdir("json_chapters")]
json_recipe_files = [os.path.join("json_chapters",file) for file in json_recipe_files]
documents = []
ids =[]

for file in json_recipe_files:
    with open(file, "r") as f:
        data = json.load(f)

        for recipe in data:
            text = f"Title: {recipe['title']} \n Instructions: {recipe['instructions']}"
            documents.append(text)
            ids.append(recipe['recipe_id'])


In [ ]:
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="recipes")

# store each document in a vector embedding database
i = 0
for recipe_id, recipe in zip(ids, documents):
  if i % 100 == 0:
    print(f"Embedding recipe: {i+1}.")
  response = ollama.embed(model="mxbai-embed-large:latest", input=recipe)
  embeddings = response["embeddings"]
  # break
  collection.add(
    ids=[str(recipe_id)],  # use the recipe ID from the JSON instead
    embeddings=embeddings[0],
    documents=[recipe]
    # can add recipe metadata here such as ingredients and the chapter title
  )
  i += 1

# Testing the Recipe Retrieval 

In [8]:
# an example input
input = "What should I cook with eggs for a garnish?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input
)
results = collection.query(
  query_embeddings=[response["embeddings"][0]],
  n_results=2
)
data = results['documents'][0][0]

In [9]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {data}. Respond to this prompt: {input}"
)

print(output['response'])

A classic dish! According to the instructions, you should cook minced onions in butter until they're cooked through but not browned. This will be the perfect garnish for your dish.

So, to summarize:

* Cook minced onions in butter until they're tender and not browned.
* Use these onions as a garnish on top of whatever main dish you've prepared.
* Break eggs over the onion topping.
* Sprinkle a small amount of grated Parrhesan cheese (a type of Parmesan) over the eggs.
* Place the dish under a hot broiler or in a fierce oven until the eggs are cooked and a slight gratin forms on top.

Enjoy your OMER-PACHA EQQS!
